# IMPORTS

In [58]:
import torch
import numpy as np

from torch import optim
from torch import nn
import torchvision
from torchvision import models
from torchvision import transforms, datasets
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

# FUCNTION

In [59]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [60]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()

        # pred = model(X)
        # loss = loss_fn(pred, y)

        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


In [61]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

# LOAD DATA

In [62]:
data = datasets.CIFAR100(root='data/', train=True, download=True)

  0%|          | 0/169001437 [00:00<?, ?it/s]

Extracting data/cifar-100-python.tar.gz to data/


In [63]:
class MyOwnCifar(torch.utils.data.Dataset):
   
    def __init__(self, init_dataset, transform=None):
        self._base_dataset = init_dataset
        self.transform = transform

    def __len__(self):
        return len(self._base_dataset)

    def __getitem__(self, idx):
        img = self._base_dataset[idx][0]
        if self.transform is not None:
            img = self.transform(img)
        return img, self._base_dataset[idx][1]

In [64]:
train_transformation = transforms.Compose([transforms.Scale(256),
                                    transforms.RandomCrop(224, padding=4),
                                    transforms.ToTensor(),
                                    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                         std=[0.229, 0.224, 0.225])])
test_transformation = transforms.Compose([transforms.ToTensor(),
                                       transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                         std=[0.229, 0.224, 0.225])])

/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:317: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  warnings.warn("The use of the transforms.Scale transform is deprecated, " +


In [65]:
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

In [66]:
train_dataset = MyOwnCifar(train_data, train_transformation)
test_dataset = MyOwnCifar(test_data, test_transformation)

In [67]:
train_loader = torch.utils.data.DataLoader(train_dataset,
                          batch_size=128,
                          shuffle=True,
                          num_workers=3)
test_loader = torch.utils.data.DataLoader(test_dataset,
                          batch_size=128,
                          shuffle=False,
                          num_workers=1)

In [68]:
len(train_loader)

313

In [69]:
len(data.classes)

100

# MODEL IMAGE NET RESNET-50

In [70]:
resnet_50 = models.resnet50(pretrained=True)

In [71]:
# print(resnet_50)

In [72]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
transform = transforms.Compose([transforms.Resize(256), transforms.CenterCrop(224), normalize, transforms.ToTensor()])

In [73]:
set_parameter_requires_grad(resnet_50, True)
resnet_50.fc = nn.Linear(2048, 100)

In [74]:
params_to_update = []
for name,param in resnet_50.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)
params_to_update

[Parameter containing:
 tensor([[ 1.0054e-02,  1.7657e-02, -7.3866e-03,  ...,  1.2985e-02,
           1.3848e-02, -1.4724e-02],
         [ 5.7044e-03,  1.6053e-02,  1.4516e-02,  ..., -1.3363e-02,
          -1.5792e-02,  1.2163e-03],
         [-5.8635e-03,  1.4690e-02, -1.5707e-02,  ...,  1.1557e-02,
          -2.1486e-02, -8.3654e-03],
         ...,
         [-1.6615e-02,  2.0777e-03,  2.3646e-03,  ...,  2.1150e-02,
          -1.3322e-02,  2.0975e-02],
         [ 1.5447e-02,  1.1661e-02, -1.7927e-02,  ...,  6.8346e-03,
          -5.8035e-03,  1.8554e-03],
         [-1.7938e-02,  1.0598e-02, -5.6472e-05,  ...,  1.9498e-02,
           1.5902e-02,  1.4927e-02]], requires_grad=True),
 Parameter containing:
 tensor([ 3.0742e-03, -2.1665e-02,  4.4363e-03, -5.6386e-03,  1.8457e-02,
          1.8140e-02,  1.5991e-02,  1.7118e-03, -2.0553e-02,  1.5676e-02,
          5.8898e-03,  3.7888e-03, -9.5141e-03,  2.1446e-02,  5.6334e-03,
          6.4768e-03, -2.1551e-02, -1.8960e-02,  1.8724e-02, -1.00

# RES NET 50 TRAINING AND VALIDATION

In [75]:
use_cuda = torch.cuda.is_available()

device = torch.device("cuda:0" if use_cuda else "cpu")
print(device)

cuda:0


In [76]:
optimizer = torch.optim.Adam(params_to_update, lr=0.001)
loss_fn = nn.CrossEntropyLoss()

In [77]:
resnet_50 = resnet_50.cuda() if use_cuda else resnet_50

In [78]:
epochs = 20
for t in range(epochs):
    print(f"Epoch {t+1}\-------------------------------")
    train(train_loader, resnet_50, loss_fn, optimizer)
    test(test_loader, resnet_50, loss_fn)
print("Done!")

Epoch 1\-------------------------------
loss: 4.648887  [    0/40000]
loss: 2.623889  [12800/40000]
loss: 2.059295  [25600/40000]
loss: 2.047357  [38400/40000]
Test Error: 
 Accuracy: 5.3%, Avg loss: 11.495063 

Epoch 2\-------------------------------
loss: 1.822363  [    0/40000]
loss: 1.704209  [12800/40000]
loss: 1.714680  [25600/40000]
loss: 1.624888  [38400/40000]
Test Error: 
 Accuracy: 5.3%, Avg loss: 14.282084 

Epoch 3\-------------------------------
loss: 1.561022  [    0/40000]
loss: 1.586663  [12800/40000]
loss: 1.507993  [25600/40000]
loss: 1.769714  [38400/40000]
Test Error: 
 Accuracy: 4.5%, Avg loss: 16.311964 

Epoch 4\-------------------------------
loss: 1.478402  [    0/40000]
loss: 1.379386  [12800/40000]
loss: 1.666318  [25600/40000]
loss: 1.714959  [38400/40000]
Test Error: 
 Accuracy: 4.7%, Avg loss: 16.439056 

Epoch 5\-------------------------------
loss: 1.772768  [    0/40000]
loss: 1.633940  [12800/40000]
loss: 1.228964  [25600/40000]
loss: 1.569689  [38400

# DATA WITH AUGMENTATION 

In [92]:
train_transformation = transforms.Compose([ 
                                    transforms.Resize(256),
                                    transforms.RandomCrop(224, padding=4),
                                     # Augmentation
                                    transforms.RandomHorizontalFlip(p=0.5),
                                    transforms.RandomVerticalFlip(p=0.5),
                                    transforms.ToTensor(),
                                    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                         std=[0.229, 0.224, 0.225])
                                    ])
test_transformation = transforms.Compose([
                                        transforms.ToTensor(),
                                       transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                         std=[0.229, 0.224, 0.225])
                                       ])

In [93]:
train_dataset_1 = MyOwnCifar(train_data, train_transformation)
test_dataset = MyOwnCifar(test_data, test_transformation)

In [94]:
big_train = torch.utils.data.ConcatDataset([train_dataset, train_dataset_1])

In [95]:
big_train_loader = torch.utils.data.DataLoader(big_train,
                          batch_size=128,
                          shuffle=True,
                          num_workers=3)
test_loader = torch.utils.data.DataLoader(test_dataset,
                          batch_size=128,
                          shuffle=False,
                          num_workers=1)

In [96]:
len(data.classes)

100

In [97]:
model = models.resnet50(pretrained=True)

set_parameter_requires_grad(model, True)

In [102]:
model = model.cuda() if use_cuda else model
    

model.fc = torch.nn.Linear(2048, 1024)
model.fc = nn.Sequential(
    torch.nn.Dropout(0.5),
    torch.nn.Linear(num_ftrs, 1024),
    torch.nn.Dropout(0.2),
    torch.nn.Linear(1024, 512),
    torch.nn.Dropout(0.2),
    torch.nn.Linear(512, 256),
    torch.nn.Dropout(0.2),
    torch.nn.Linear(256, 128),
    torch.nn.Dropout(0.2),
    torch.nn.Linear(128, len(data.classes))
)

model.fc = model.fc.cuda() if use_cuda else model.fc

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=1e-2, momentum=0.9, weight_decay=5e-4)

In [103]:
params_to_update = []
for name,param in model.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)
params_to_update

[Parameter containing:
 tensor([[ 0.0024, -0.0044,  0.0054,  ...,  0.0119, -0.0183, -0.0098],
         [ 0.0086,  0.0173, -0.0199,  ..., -0.0137,  0.0153,  0.0071],
         [-0.0205, -0.0035,  0.0099,  ...,  0.0085,  0.0214, -0.0206],
         ...,
         [ 0.0005,  0.0216,  0.0119,  ..., -0.0185,  0.0143, -0.0197],
         [ 0.0039,  0.0133,  0.0146,  ..., -0.0215,  0.0138,  0.0082],
         [-0.0087, -0.0082,  0.0148,  ..., -0.0093, -0.0160, -0.0195]],
        device='cuda:0', requires_grad=True), Parameter containing:
 tensor([ 0.0214, -0.0190,  0.0081,  ...,  0.0141,  0.0087,  0.0163],
        device='cuda:0', requires_grad=True), Parameter containing:
 tensor([[ 0.0298, -0.0158, -0.0018,  ..., -0.0002,  0.0073, -0.0087],
         [ 0.0312,  0.0304, -0.0059,  ...,  0.0132,  0.0004,  0.0160],
         [-0.0061, -0.0268, -0.0196,  ...,  0.0051, -0.0305,  0.0149],
         ...,
         [-0.0187,  0.0158, -0.0068,  ..., -0.0047, -0.0224, -0.0086],
         [-0.0132,  0.0265, -0.0

In [104]:
epochs = 20
for t in range(epochs):
    print(f"Epoch {t+1}\-------------------------------")
    train(big_train_loader, model, loss_fn, optimizer)
    test(test_loader, model, loss_fn)
print("Done!")

Epoch 1\-------------------------------
loss: 4.602438  [    0/80000]
loss: 4.548287  [12800/80000]
loss: 3.832710  [25600/80000]
loss: 3.316126  [38400/80000]
loss: 3.097594  [51200/80000]
loss: 2.916045  [64000/80000]
loss: 2.582769  [76800/80000]
Test Error: 
 Accuracy: 4.8%, Avg loss: 8.917118 

Epoch 2\-------------------------------
loss: 2.740299  [    0/80000]
loss: 2.861262  [12800/80000]
loss: 2.594682  [25600/80000]
loss: 2.676989  [38400/80000]
loss: 2.993343  [51200/80000]
loss: 2.719795  [64000/80000]
loss: 2.911940  [76800/80000]
Test Error: 
 Accuracy: 5.4%, Avg loss: 10.819133 

Epoch 3\-------------------------------
loss: 2.593208  [    0/80000]
loss: 2.562203  [12800/80000]
loss: 2.486384  [25600/80000]
loss: 2.410337  [38400/80000]
loss: 2.431212  [51200/80000]
loss: 2.502540  [64000/80000]
loss: 2.527037  [76800/80000]
Test Error: 
 Accuracy: 5.4%, Avg loss: 10.727769 

Epoch 4\-------------------------------
loss: 2.304221  [    0/80000]
loss: 2.671883  [12800/80